In [14]:
import requests

class Asset:
    """
    A class representing a depreciable asset.

    Attributes:
    -----------
    cost : float
        The initial cost of the asset.
    salvage_value : float
        The estimated value of the asset at the end of its useful life.
    useful_life : int
        The estimated number of periods over which the asset will be depreciated.

    Methods:
    --------
    straight_line_amortization():
        Calculates the depreciation expense using the straight-line method.
    double_declining_balance_amortization():
        Calculates the depreciation expense using the double-declining balance method.
    annuity_amortization(risk_factor):
        Calculates the depreciation expense using the annuity method.

    """
    def __init__(self, cost, salvage_value, useful_life):
        """
        Initializes a new instance of the Asset class.

        Parameters:
        -----------
        cost : float
            The initial cost of the asset.
        salvage_value : float
            The estimated value of the asset at the end of its useful life.
        useful_life : int
            The estimated number of periods over which the asset will be depreciated.

        """
        self.cost = cost
        self.salvage_value = salvage_value
        self.useful_life = useful_life
        
    def get_yield_curve(self):
        """
        Retrieves the Canadian Interest Rate Swap (IRS) curve from the Bank of Canada API.

        Returns:
        --------
        list of float:
            The IRS rates for various maturities, expressed as annualized percentages.

        Raises:
        ------
        requests.exceptions.RequestException:
            If there is an error retrieving the data from the API.

        """
        # Get the yield curve rates from the Bank of Canada API
        url = 'https://www.bankofcanada.ca/valet/observations/IRSLTDTY/json'
        response = requests.get(url)
        data = response.json()
#         series = data['observations']
        
#         # Extract the interest rates from the API response
#         yield_curve = []
#         for point in series:
#             yield_curve.append(point['IRSLTDTY'])
            
        # return yield_curve
        return data
    
    def straight_line_amortization(self):
        """
        Calculates the depreciation expense using the straight-line method.

        Returns:
        --------
        float:
            The depreciation expense for each period, calculated using the straight-line method.

        """
        depreciation_per_period = (self.cost - self.salvage_value) / self.useful_life
        return depreciation_per_period
    
    def double_declining_balance_amortization(self):
        """
        Calculates the depreciation expense using the double-declining balance method.

        Returns:
        --------
        float:
            The depreciation expense for each period, calculated using the double-declining balance method.

        """
        depreciation_rate = 2 / self.useful_life
        depreciation_per_period = self.cost * depreciation_rate
        while depreciation_per_period > (self.cost - self.salvage_value):
            depreciation_rate -= 0.01
            depreciation_per_period = self.cost * depreciation_rate
        return depreciation_per_period
    
    def annuity_amortization(self, risk_factor):
        """
        Calculates the depreciation expense using the annuity method.

        Parameters:
        -----------
        risk_factor : float
            A factor added to the IRS rates to account for the asset's specific risk profile.

        Returns:
        --------
        float:
            The depreciation expense for each period, calculated using the annuity method.

        Raises:
        ------
        requests.exceptions.RequestException:
            If there is an error retrieving the data from the Bank of Canada API.

        """
        yield_curve = self.get_yield_curve()
        annuity_factor = 0
        for i in range(1, self.useful_life + 1):
            discount_factor = 1 / ((1 + yield_curve[i-1] + risk_factor) ** i)
            annuity_factor += discount_factor
        depreciation_per_period = (self.cost - self.salvage_value) / annuity_factor
        return depreciation_per_period


In [15]:
a = Asset(10000, 2000, 10)

In [16]:
a.get_yield_curve()

{'message': 'Series IRSLTDTY not found.',
 'docs': 'https://www.bankofcanada.ca/valet/docs'}